In [2]:
import ebooklib
import re
import nltk
import pandas as pd

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import pos_tag, word_tokenize

from ebooklib import epub
from bs4 import BeautifulSoup as bs

In [3]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /home/vdsukhov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/vdsukhov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vdsukhov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
eb = epub.read_epub('./data/dune_book.epub') # speicify path for your book here
items = list(eb.get_items_of_type(ebooklib.ITEM_DOCUMENT))
items = [item for item in items if 'chapter' in item.get_name()]

/home/vdsukhov/mambaforge/envs/wc/lib/python3.10/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
/home/vdsukhov/mambaforge/envs/wc/lib/python3.10/site-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


In [7]:
buffer = []
for it in items:
    sp = bs(it.get_body_content(), 'html.parser')
    text = " ".join([p.get_text() for p in sp.find_all('p')])
    buffer.append(text)

dune = " ".join(buffer)
print(dune[:100])

A beginning is the time for taking the most delicate care that the balances are correct. This every 


In [8]:
dune = dune.split()
dune = " ".join([re.sub(r'\W+', '', w) for w in dune])
print(dune[:100])

A beginning is the time for taking the most delicate care that the balances are correct This every s


In [9]:
tokens = word_tokenize(dune)
tokens = pos_tag([t.lower() for t in tokens])
print(tokens[:5])

[('a', 'DT'), ('beginning', 'NN'), ('is', 'VBZ'), ('the', 'DT'), ('time', 'NN')]


In [10]:
wnl = WordNetLemmatizer()
lemmas = [wnl.lemmatize(token, pos=tag[0].lower()) if tag[0].lower() in ['a', 'n', 'v'] else wnl.lemmatize(token) for token, tag in tokens]
print(lemmas[:5])

['a', 'beginning', 'be', 'the', 'time']


In [11]:
stop_words = set(stopwords.words('english'))
lemmas = [l.upper() for l in lemmas if l not in stop_words]
print(lemmas[:5])

['BEGINNING', 'TIME', 'TAKE', 'DELICATE', 'CARE']


In [12]:
freqd = {w: lemmas.count(w) for w in set(lemmas)}
df = pd.DataFrame({'word': freqd.keys(), 'freq': freqd.values()})
df = df.sort_values(by='freq', ascending=False, ignore_index=True)
df.head(10)

,word,freq
0,SAY,2463
1,PAUL,1652
2,JESSICA,845
3,THINK,755
4,ONE,633
5,KNOW,620
6,BARON,568
7,SEE,562
8,DUKE,560
9,COULD,511


In [14]:
df.to_csv("./data/dune_words.csv")